In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from data_management.indicator_manager import IndicatorManager



eur_1min_source = Path('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1min/EUR_USD.parquet')
df_1min = pd.read_parquet(eur_1min_source)

dataset_manager = DatasetManager()
indicator_manager = IndicatorManager()
new = dataset_manager.aggregate_timeframe(df=df_1min, source_timeframe='1min', target_timeframe='15min')
new.dropna(inplace=True)
min5_indics = indicator_manager.calculate_indicators(df=new)
min5_indics.dropna(inplace=True)
min5_indics.to_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_indics_not_norm.parquet')


In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from data_management.indicator_manager import IndicatorManager

min5_not_norm = pd.read_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/EUR_USD.parquet')
from visualization.chart_manager import ChartManager
chart_manager = ChartManager()
start_time = pd.Timestamp('2024-01-01').tz_localize('UTC')
end_time = pd.Timestamp('2024-10-31').tz_localize('UTC')
chart_manager.chart(min5_not_norm, start_time, end_time)

In [ ]:
min5_not_norm = pd.read_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/EUR_USD.parquet')
min5_not_norm